# Texas 2k Case and ACOPF

In [1]:
import numpy as np

import ams

In [2]:
!ams misc --version

Python   3.12.0
ams      1.0.9
andes    1.9.3
numpy    2.1.2
cvxpy    1.5.3
solvers  CLARABEL, ECOS, ECOS_BB, OSQP,
         PIQP, SCIP, SCIPY, SCS


In [3]:
%matplotlib inline

In [4]:
ams.config_logger(stream_level=20)

First, we can read the MATPOWER format case file into a Python dictionary.

In [5]:
mpc = ams.io.matpower.m2mpc("./Texas2k_series24_case3_2024summerpeak.m")

 Case saved by PowerWorld Simulator, version 23, build date October 25, 2023


In [6]:
ngen = mpc["gencost"].shape[0]

This case gives a piecewise lienar cost function with two pairs of points.

For simplification, it is manually mapped to a quadratic cost function with linear term and constant term.

In [7]:
gencost2 = np.concatenate((mpc['gencost'][:, 0:4], np.zeros((ngen, 4))), axis=1)

gencost2[:, 0] = 2  # type of cost function
gencost2[:, 3] = 3  # number of coefficients
gencost2[:, 4] = 0.0  # quadratic coefficient
gencost2[:, 5] = mpc["gencost"][:, 7]  # linear coefficient
gencost2[:, 6] = mpc["gencost"][:, 5]  # constant term

mpc2 = mpc.copy()
mpc2['gencost'] = gencost2

In [8]:
sp2 = ams.system.System()
ams.io.matpower.mpc2system(mpc2, sp2)

sp2.setup()

Zero Line parameters detected, adjusted to default values: rate_b, rate_c, amax, amin.
System set up in 0.0113 seconds.


True

In [9]:
sp3 = ams.load("./Texas2k_series24_case3_2024summerpeak_0_33.RAW",
               setup=False,
               no_output=False,)

Parsing input file "./Texas2k_series24_case3_2024summerpeak_0_33.RAW"...
Input file parsed in 0.5889 seconds.


In [10]:
stg_mp = set(sp2.StaticGen.get_all_idxes())
stg_raw = set(sp3.StaticGen.get_all_idxes())
stg_equal = stg_mp == stg_raw
stg_equal

True

The two cases have idential generators, so we can add generator costs to the PSSE read case.

In [11]:
for index, row in sp2.GCost.cache.df_in.iterrows():
    sp3.add(model='GCost', **row)

In [12]:
sp3.setup()

Zero Line parameters detected, adjusted to default values: rate_b, rate_c.
System set up in 0.0101 seconds.


True

We can scale down the total load for better convergence if necessary.

In [13]:
p0 = sp3.StaticGen.get(src='p0', attr='vin', idx=sp3.StaticGen.get_all_idxes())
sp3.StaticGen.alter(src='p0', attr='vin', idx=sp3.StaticGen.get_all_idxes(),
                    value=1 * p0)

True

In [14]:
sp3.DCOPF.run(solver='PIQP', ignore_dpp=True)

Building system matrices
Parsing OModel for <DCOPF>
Evaluating OModel for <DCOPF>
Finalizing OModel for <DCOPF>
<DCOPF> initialized in 0.5957 seconds.
<DCOPF> solved as optimal in 0.0401 seconds, converged in 13 iterations with PIQP.
Report saved to "Texas2k_series24_case3_2024summerpeak_0_33_out.txt" in 0.2140 seconds.


True

In [15]:
ams.io.xlsx.write(sp3, 'Texas2k.xlsx', overwrite=True)

xlsx file written to "Texas2k.xlsx"


True

In [16]:
ams.io.json.write(sp3, 'Texas2k.json', overwrite=True)

JSON file written to "Texas2k.json"


True

In [17]:
sp3.ACOPF.run()

Parsing OModel for <ACOPF>
Evaluating OModel for <ACOPF>
Finalizing OModel for <ACOPF>
<ACOPF> initialized in 0.8103 seconds.
<ACOPF> solved in 34.5884 seconds, converged in 37 iterations with PYPOWER-PIPS.
Report saved to "Texas2k_series24_case3_2024summerpeak_0_33_out.txt" in 0.4004 seconds.


In [18]:
sp3.ACOPF.export_csv()

'Texas2k_series24_case3_2024summerpeak_0_33_ACOPF.csv'